In [2]:
!pip install -r requirements.txt

  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached cachetools-4.2.1-py3-none-any.whl (12 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached cmake-3.18.4.post1-py3-none-manylinux1_x86_64.whl (17.7 MB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached facenet_pytorch-2.5.1-py3-none-any.whl (1.9 MB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached google_auth-1.28.1-py2.py3-none-any.whl (136 kB)
  Using cached google_auth_oauthlib-0.4.4-py2.py3-none-any.whl (18 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.32.0-cp38-cp38-manylinux2014_x86_64.whl (3.8 MB)
  Using cached h5py-2.10.0-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
  Usi

# 1. Crosscut Class 구현

### - 1. 초기화 함수

In [1]:
def __init__(self, dist_obj, video_path, output_path):
    # 무대 영상들이 저장된 폴더 경로
    self.videos_path = video_path
    # 만든 교차편집 영상을 저장할 경로와 파일명
    self.output_path = output_path
    # 무대 영상 중 가장 짧은 영상의 길이(초)로 클래스 내부에서 자동 계산
    # 1000.0 : 초기값
    self.min_time = 1000.0
    # 무대 영상 개수
    video_num = len(os.listdir(self.videos_path))
    # 각 무대 영상의 시작점
    self.start_times = [0] * video_num
    # 영상을 비교할 길이(초)
    self.window_time = 10
    # 영상 전환이 너무 자주 일어나면 영상이 산만해질 수 있으므로
    # 일정 시간 동안 전환을 방지
    self.padded_time = 4
    # Random거리, 얼굴기반 Landmark, 자세 기반 Landmark 중 하나를 정해 할당
    self.dist_obj = dist_obj
    # 교차편집에 사용할 음성
    self.audioclip = None
    # 시작점이 정렬된 무대 영상 저장
    self.extracted_clips_array = []
    # 교차편집으로 내보낼 짧은 영상들 저장
    self.con_clips = []

### - 2. 무대 영상들의 시작점을 정렬하는 함수

In [2]:
def video_alignment(self):
    for i in range(len(os.listdir(self.videos_path))):
        video_path = os.path.join(self.videos_path, 
                                 sorted(so.listdir(self.videos_path))[i])
        clip = VidoeFileClip(video_path)
        clip = clip.subclip(self.start_times[i], clip.duration)
        
        if self.min_time > clip.duration:
            self.audioclip = clip.audio
            self.min_time = clip.duration
        self.extracted_clips_array.append(clip)
    print('LOGGER-- {} Vidoe Will Be Mixed'.format (len(self.extracted_clips_array)))

### - 3. 다음 영상을 선택하는 함수

In [3]:
def select_next_clip(sefl, t, current_idx):
    # 1. 거리 측정에 필요한 변수 초기화하기
    cur_t = t
    next_t = min(t + self.window_time, self.min_time)
    
    reference_clip = self.extracted_clips_array[current_idx].subclip(cur_t, next_t)
    d = float('Inf')
    cur_clip = None
    min_idx = (current_idx + 1) % len(self.extracted_clips_array)
    
    # 2. 비교 영상들 과 현재 영상의 거리 측정
    for video_idx in range(len(self.extracted_clips_array)):
        if video_idx == current_idx:
            continue
        clip = self.extracted_clips_array[video_idx].subclip(cur_t, next_t)
        cur_d, plus_frame = self.dist_obj.distance(reference_clip, clip)
        print(current_idx, video_idx,cur_d, cur_t + plus_frame)
        if d > cur_d:
            d = cur_d
            min_idx = video_idx
            next_t = cur_t + plus_frame
            cur_clip = reference_clip.subclip(0, plus_frame)
            
    # 3. 다음 교차편집 지점 전까지 현재 영상 저장하기
    if cur_clip:
        clip = cur_clip
    else:
        clip = reference_clip
    self.con_clips.append(clip)
    
    # 4. 현재 시간을 갱신하고 다음에 사용할 영상 인덱스 반환하기
    t = next_t
    return t, min_idx

### - 4. 선택한 영상의 padding을 추가하는 함수

In [4]:
def add_padding(sef, t, next_idx):
    print('idx : {}'.format(next_idx))
    pad_clip = self.extracted_clips_arra[next_idx].subclip(t,min(self.min_time, t + self.padded_time))
    self.con_clips.append(pad_clip)
    
    t = min(self.min_time, t + self.padded_time)
    return t, next_idx    

### - 5. 교차편집을 저장하는 함수

In [5]:
def write_video(self):
    final_clip = concatenate_videoclips(self.con_clips)
    if self.audioclip != None:
        print("Not None")
        final_clip.audio= self.audioclip
    final_clip.write_videofile(self.output_path)
    return final_clip

### - 6. 교차편집을 생성하는 함수

In [6]:
def generate_video(self):
    # 1. 영상 전처리하기
    self.video_alignment()
    t = 3
    corrent_idx = 0
    self.con_clips.append(self.extracted_clips_array[current_idx].subclip(0, min(t, int(self.min_time))))
    
    # 2. 노래 끝까지 교차편집 영상 만들기
    while t < int(self.min_time):
        t, min_idx = self.select_next_clip(t, current_idx)
        t, current_idx = self.add_padding(t,min_idx)
        
    # 3. 교차편집 결과 영상 저장하기
    final_clip = self.write_video()
    return final_clip

### - 7. 랜덤으로 거리와 시간을 생성하는 클래스

In [8]:
class RandomDistance:
    def distance(self, reference_clip, compare_clip):
        dur_end = min(reference_clip.duration, compare_clip.duration)
        return random.randrange(1, 100), min(dur_end, random.randrange(3, 7))

# 2. FaceDistance Class 구현

### - 1. 초기화 함수

In [9]:
def __init__(self, shape_predictor_path, face_embedding_penalty = None):
    # skip_frame_rate : 두 영상 간의 거리를 계산할 Frame 단위
    self.skip_frame_rate = 4
    # 얼굴이 가장 비슷한 지점에서 영상 전환
    self. minimax_frames = 5
    # shape_predictor_68_face_landmarks.dat 모델 경로
    self.shape_predictor = shape_predictor_path
    # face_embedding_penalty : 두 영상에서 얼굴이 다를 때 더해지는 최대 penalty 값
    self.face_embedding_penalty = face_embedding_penalty

### - 2. 얼굴의 Landmarks를 추출하는 함수

In [10]:
def extract_landmark(self, reference_clip, compare_clip):
    # 1. 영상 저장 및 face landmark detect model 불러오기
    # 비교할 두 영상을 self.clips에 저장
    self.clips = [reference_clip, compare_clip]
    # 얼굴 자체의 위치를 사각형으로 잡아주는 모델
    detector = dlib.get_frontal_face_detector()
    # 얼굴 위치가 잡힌 사각형 내에서 얼굴의 landmark를 구해주는 모델
    predictor = dlib.shape_predictor(self.shape_predictor)
    clips_frame_info = []
    
    for clip in self.clips:
        # 2. 각 영상의 정보를 저장하기 위해 loop마다 초기화하기
        # 현재 확인중인 frame 번호. 현재 frame이 몇 번째인지 알려줌
        i = 0
        # 각 영상의 Frame 정보 담기
        every_frame_info = []
        
        while True:
            # 3. 각 영상에서 face Landmark 얻기
            # get_frame : 해당 시간의 Frame을 이미지로 바꿔서 Frame에 반환하는 Moviepy 제공 함수
            frame = clip.get_frame(i * 1.0 / clip.fps)
            # 다음 loop에서 사용할 frame index 저장
            i += self.skip_frame_rate
            # 영상의 끝부분까지 Facial Landmark를 얻기 위한 while문
            # clip.fps : 1초당 들어 있는 Frame 수
            # 1.0 / clip.fps : 한 Frame이 차지하는 시간
            # i * 1.0 . clip.fps : 현재 확인하고 있는 Frame의 시간대
            if(i*1.0/clip.fps) > clip.duration:
                break
            
            # 이미지가 작을수록 계산 속도가 빠르나 정확도가 낮아짐
            frame = imutils.resize(frame, width = 800)
            # 흑백사진을 이용하면 계산 속도가 빨라짐
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # detector로 얼굴의 위치를 사각형으로 잡아줌
            rects = detector(gray, 0)
            
            # 4. 얻은 face Landmark를 가공해서 every_frame_info에 저장
            # 얼굴 개수 파악. 하나라도 있으면 facial landmark 계산, 없으면 빈 list 반환
            if len(rects) > 0:
                max_width = 0
                #계산의 효율성을 위해 얼굴 중에 가장 큰 얼굴만 detect
                max_rect = None
                # max_width를 통해 가장 너비가 큰 사각형을 찾고, max_rect에 해당 얼굴 저장
                for rect in rects:
                    if int(rect[0].width()) > max_width:
                        max_rect = rect
                # 가장 크게 확대된 얼굴의 facial landmark를 구함
                shape = predictor(gray, max_rect)
                # 구해진 facial landmark 결과를 계산 용이성을 위해 numpy로 변환
                shape = face_utils.shape_to_np(shape)
                # 해당 frame의 facial landmark 정보를 every_frame_info에 추가
                every_frame_info.append(shape)
            else:
                every_frame_info.append([])
                
        # 5. 영상 frame별 landmark 정보를 clips_frame_info에 저장
        # 영상의 모든 frame의 facial landmark 정보가 담긴 every_frame_info를 clips_frame_info에 저장
        clips_frame_info.append(np.array(every_frame_info))
        
    cv2.destroyAllWindows()
    return clips_frame_info                                

### - 3. 얼굴의 Embedding 값의 Cosine Distance를 계산하는 함수

In [11]:
# 얼굴 Embedding 값으로 penalty를 주는 함수
def embedding_cosine_distance(self, reference_frame, compare_frame):
    face_detector = MTCNN(select_largest = True)
    embed_model = InceptionResnetV1(pretrained = 'vggface2').eval()
    
    reference_frame = np.array(reference_frame)
    compare_frame = np.array(compare_frame)
    try :
        reference_frame_detected = face_detector(reference_frame)
        compare_frame_detected = face_detector(compare_frame)
    except:
        cosine_dist = 1
        return cosine_dist
    
    reference_frame_embed = embed_model(reference_frame_detected.unsqueeze(0)).detach().numpy()
    compare_frame_embed = embed_model(compare_frame_detected.unsqueeze(0)).detach().numpy()
    reference_frame_embed = np.squeeze(reference_frame_embed)
    compare_frame_embed = np.squeeze(compare_frame_embed)
    cosine_dist = 1 - np.dot(reference_frame_embed, compare_frame_embed) / (np.linalg.nrom(reference_frame_embed) * np.linalg.norm(compare_frame_embed))

    return cosine_dist

### - 4. 두 영상의 각 Frame 간 거리를 측정하는 함수

In [12]:
def get_all_frame_distance(self, clips_frame_info, min_size):
    # dist_arr에는 같은 시간에 두 영상의 거리를 각 frame별로 저장
    dist_arr = []
    # clip_frame_info에서 각 영상이 가진 frame 정보 순회
    for i in range(min_size - 1):
        # 두 영상 모두에 facial landmark 정보가 있을 때만 거리 계산
        if len(clips_frame_info[0][i]) > 0 and (clips_frame_info[1][i+1]) > 0:
            # 양쪽 눈 끝만 사용해도 특징을 표현할 수 있음
            l = 36
            r = 45
            left_eye = (
                (clips_frame_info[0][i][l][0] - clips_frame_info[1][i+1][l][1]) ** 2 +\
                (clips_frame_info[0][i][l][1] - clips_frame_info[1][i+1][l][1]) ** 2
            ) ** 0.5
            right_eye = (
                (clips_frame_info[0][i][r][0] - clips_frame_info[1][i+1][r][1]) ** 2 +\
                (clips_frame_info[0][i][r][1] - clips_frame_info[1][i+1][r][1]) ** 2
            ) ** 0.5
            # 각 frame별 두 얼굴 사이의 거리를 dist_arr에 붙여넣음
            total_diff = left_eye + right_eye
            dist_arr.append(total_diff)
        else:
            dist_arr.append(None)
    return dist_arr

### - 5. 얼굴 기반 영상의 거리를 계산하는 함수

In [13]:
def distance(self, reference_clip, compare_clip):
    # 1. 거리 계산에 필요한 정보 얻기
    # extract_landmark 함수를 이용해 각 영상마다 frame들의 facial landmark 정보가 담긴 clips_frame_info 변수 구하기
    clips_frame_info = self.extract_landmark(reference_clip, compare_clip)
    # min 함수로 더 짧은 영상을 기준으로 size 지정
    min_size = min(len(clips_frame_info[0]), len(clips_frame_info[1]))
    # get_all_frame_distance 함수를 통해 모든 frame의 거리 정보가 담긴 dist_arr을 얻음
    dist_arr = self.get_all_frame_distance(clips_frame_info, min_size)
    # 최소거리와, 최소거리를 가지는 index를 저장하기 위해 초기화
    clips = [reference_clip, compare_clip]
    minimax_frames = self.minimax_frames
    min_diff = np.float('Inf')
    min_idx = 0
    # 2. 최소 거리가 되는 영상과 시간 찾기
    # self.minimax_frames 개수만큼의 frame 확인
    for i in range(min_size - (minimax_frames - 1)):
        # 해당 frame 앞뒤의 거리 정보를 다 비교해야 함
        # 첫 번째 frame에서는 이전 frame이 없음
        # 이전 frame이 없는 frame일 경우에는 시작점을 0으로 잡도록 함
        # 그렇지 않은 경우에는 해당 frame index에서 minimax_frame값을 빼서 앞뒤 frame을 거리 계산에 고려
        start_minmax_idx = 0 if (i - minimax_frames) < 0 else i - minimax_frames
        # 해당 frame과 minimax_frames 만큼 앞뒤에 얼굴 자체가 없는(None) 경우가 있는지 확인
        if(None not in dist_arr[start_minimax_idx :i + minimax_frames]):
            # np.max 함수를 통해 해당 범위 내의 최댓값(tmp_max)을 얻음
            # 해당 영상의 대표값이 되는 최댓값이 최소거리(min_diff)보다 작은지 판단
            tmp_max = np.max(dist_arr[start_minmax_idx:i + minimax_frames])
            if min_diff > tmp_max:
                min_diff = tmp_max
                min_dix = i
                
    # 3. face embedding penalty 추가하기
    if self.face_embedding_penalty != None and min_diff < np.float('Inf'):
        ref_frame = reference_clip.get_frame(min_idx * 1.0 / reference_clip.fps)
        frame = compare_clip.get_frame(min_idx * 1.0 / compare_clip.fps)
        cosine_dist = self.embedding_cosine_distance(ref_frame, frame)
        min_diff += cosine_dist * self.face_embedding_penalty
    # 4. 두 영상 간의 최소 거리 정보 return
    return min_diff, (min_idx * self.skip_frame_rate) / self.clips[0].fps

# 3. PoseDistance Class 구현

### - 1. 초기화 함수

In [15]:
def __init__(self):
    # 두 영상 간의 거리를 계산할 frame 단위
    self.SKIP_FRAME_RATE = 10
    # 거리를 계산할 때 확인하는 앞뒤 frame 수
    self.MINIMAX_FRAME = 4
    # pytorch pose estimation - alphapose -> faster R-CNN모델 사용
    self.model = models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    self.model.eval()
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

### - 2. 가수의 위치를 파악하는 함수

In [19]:
def extract_boxes(self, reference_clip, compare_clip):
    # 1. 변수 초기화
    self.clips =[reference_clip, compare_clip]
    clips_frame_info = []
    for clip in self.clips:
        # 2. 각 영상의 정보를 저장하기 위해 loop마다 초기화하기
        i = 0
        every_frame_info = []
        # 영상의 끝 부분까지 pose box를 얻기 위한 while문
        while True :
            # 3. Faster R-CNN을 이용해 물체 판별하기
            # 다음 loop에서 사용할 frame index를 저장
            i += self.SKIP_FRAME_RATE
            # clip.fps : 1초당 들어있는 frame 수
            # 1.0 / clip.fps : 한 frame이 차지하는 시간
            # i * 1.0 / clip.fps : 현재 확인중인 frame의 시간대
            # 현재 확인 중인 frame의 시간(i*1.0/clip.fps)이 영상의 전체 시간(clip.duration)보다 크면 break
            if (i * 1.0 / clip.fps) > clip.duration:
                break
            
            # get_frame : Moviepy 제공 함수. 해당 시간의 frame을 이미지로 바꿔서 frame에 저장
            frame = clip.get_frame(i * 1.0 / clip.fps)
            # frame 크기 조정 함수
            frame = imutils.resize(frame, width = 640)
            # 원래 get_frame으로 얻은 이미지는 0~255 사이 수를 가짐
            # alphapose에 입력되는 이미지는 0~1 사이 수를 가짐
            # 그러므로 모든 수를 255로 나눠줘야 함
            frame = frame / 255
            # get_frame으로 얻은 이미지는 height, width, channel 순서임
            frame = np.transpose(frame, (2, 0, 1))
            # numpy 배열을 pytorch에서 사용 가능하도록 바꿔줌
            x = [torch.from_numpy(frame).float()]
            # input x를 faster r-cnn 모델에 넣어서 결괏값을 받음
            # GPU는 predictions가 여러개가 있을 수 있음
            # CPU는 predictions가 list 안에서 하나의 결과만을 가짐
            # 그러므로 첫번째 predictions를 prediction으로 저장해줌
            predictions = self.model(x)
            prediction = predictions[0]
            
            # 4. 판별정보 재가공
            # 두개의 list에서 같은 index의 값을 묶어주는 zip 함수
            each_box_list = zip(prediction['boxes'].tolist(), prediction['labels'].tolist(), prediction['scores'].tolist())
            # filter : 첫번째 argument로 함수를 받고, 두번째 argument로 list를 받아서
            #          함수의 조건에 맞는 list 요소만 반환함
            filtered_box_list = filter(lambda x: x[1] == 1 and x[2] >= 0.95, each_box_list)
            # map : 첫번째 argument로 함수를 받고, 두번째 argument로 list를 받아
            #       각 list의 요소에 함수를 하나씩 반영
            filtered_center_dot_list = list(map(lambda x: [(x[0][0] + x[0][2]) / 2, (x[0][1] + x[0][3]) / 2], filtered_box_list))
            # list 정렬
            sorted_dot_list = sorted(filtered_center_dot_list, key = lambda x: x[0])
            
            # 5. 재가공한 정보 every_frame_info에 저장
            every_frame_info.append(sorted_dot_list)
            
        # 6. 영상 frame별 landmark 정보 clips_frame_info에 저장
        clips_frame_info.append(np.array(every_frame_info))
    return clips_frame_info
                             

### - 3. 두 영상의 frame간 거리를 계산하는 함수

In [20]:
def get_all_frame_distance(self, clips_frame_info, min_size):
    dist_arr = list()
    for i in range(min_size):
        # 두 영상 모드에서 pose 정보가 있을 ㄸ만 거리를 계산
        if len(clips_frame_info[0][i]) > 0 and len(clips_frame_info[1][i]) > 0:
            # 같은 시간을 기준으로 두 영상 frame의 pose boxes 정보를 얻음
            ref_frame_dots = clips_frame_info[0][i]
            compare_frame_dots = clips_frame_info[1][i]
            # 둘 다 box가 있어 둘 사이의 거리를 계산할 수 있을 때만 거리를 더하도록 점의 최소 개수 구하기
            min_dot_num = min(len(ref_frame_dots), len(compare_frame_dots))
            # 두 영상의 점의 개수 차이를 dot_num_diff에 저장
            dot_num_diff = abs(len(ref_frame_dots) - len(compare_frame_dots))
            # penalty : 영상 중 하나가 점이 없어서 거리를 계산할 수 없을 때 더하는 가장 먼 거리의 값
            # 영상의 끝과 끝의 대각선이 가장 긴 거리
            penalty = ((self.clips[0].w ** 2 + self.clips[0].h ** 2) ** 0.5) * abs(len(ref_frame_dots) - len(compare_frame_dots))
            # 두 영상 중 하나에 점이 없으면 해당 부분에 penalty
            total_diff = penalty * dot_num_diff
            # 두 영상에서 가까운 x좌표끼리의 거리를 계산해서 total_diff에 추가
            for dot_idx in range(min_dot_num):
                total_diff += ((ref_frame_dots[dot_idx][0] - compare_frame_dots[dot_idx][0]) ** 2 + (ref_frame_dots[dot_idx][1] - compare_frame_dots[dot_idx][1]) ** 2) ** 0.5
                # 두 영상 frame간의 거리를 dist_arr에 추가
                dist_arr.append(total_diff)
            else :
                dist_arr.append(None)
        return dist_arr

### - 4. 거리 측정 함수

In [21]:
def distance(self, reference_clip, compare_clip):
    # 1. 거리 계산에 필요한 정보들 먼저 얻기
    # extract_boxes 함수를 이용해 각 영상마다 frame들의 box정보가 담긴 clips_frame_info 변수를 구함
    clips_frame_info = self.extract_boxes(reference_clip, compare_clip)
    # min 함수로 더 짧은 영상을 기준으로 size를 지ㅇ
    min_size = min(len(clips_frame_info[0]), len(clips_frame_info[1]))
    # 모든 frame의 거리 정보가 담긴 dist_arr을 얻음
    dist_arr = self.get_all_frame_distance(clips_frame_info, min_size)
    # 최소 거리와, 최소 거리를 가지는 index를 저장하기 위해 초기화
    min_diff = np.float('Inf')
    min_idx = 0
    
    # 2. 최소 거리가 되는 영상과 시간 찾기
    # 끝부분은 self.MINIMAX_FRAME만큼의 frame을 확인하지 못하므로 건너뜀
    for i in range(min_size - (self.MINIMAX_FRAME - 1)):
        # 첫번째 frame에서는 이전 frame이 없음
        # 이전 frame이 없는 frame일 경우 시작점을 0으로 잡음
        start_minmax_idx = 0 if (i - self.MINIMAX_FRAME) < 0 else i - self.MINIMAX_FRAME
        # 해당 frame과 self.MINIMAX_FRAME만큼의 앞뒤에 얼굴 자체가 없는(None) 경우가 있는지 확인
        if (None not in dist_arr[start_minmax_idx : i + self.MINIMAX_FRAME]):
            # np.max 함수를 통해 해당 범위 내의 최댓값(tmp_max)을 얻음
            # 최댓값이 최소 거리(min_diff)보다 작은지 판단
            tmp_max = np.max(dist_arr[i 
                                     : i + self.MINIMAX_FRAME])
            if min_diff > tmp_max:
                min_diff = tmp_max
                min_idx = i
        
    # 3. 두 영상 간의 최소 거리 정보 반환
    return min_diff, (min_idx * self.SKIP_FRAME_RATE) / reference_clip.fps

# --------------------------------------------------------------------------------------------------------------------

## Crosscut Class 통합

In [1]:
class Crosscut:
    def __init__(self, dist_obj, video_path, output_path):
        self.videos_path = video_path
        self.output_path = output_path
        self.min_time = 1000.0
        video_num = len(os.listdir(self.videos_path))
        self.start_times = [0] * video_num
        self.window_time = 10
        self.padded_time = 4
        self.dist_obj = dist_obj
        self.audioclip = None
        self.extracted_clips_array = []
        self.con_clips = []
        
    def video_alignment(self):
        for i in range(len(os.listdir(self.videos_path))):
            video_path = os.path.join(self.videos_path, sorted(os.listdir(self.videos_path))[i])
            clip = VideoFileClip(video_path)
            clip = clip.subclip(self.start_times[i], clip.duration)
            if self.min_time > clip.duration:
                self.audioclip = clip.audio
                self.min_time = clip.duration
            self.extracted_clips_array.append(clip)
        print('Logger-- {} Video Will Be Mixed'.format(len(self.extracted_clips_array)))
        
    def select_next_clip(self, t, current_idx):
        cur_t = t
        next_t = min(t + self.window_time, self.min_time)
        
        reference_clip = self.extracted_clips_array[current_idx].subclip(cur_t, next_t)
        d = float('Inf')
        cur_clip = None
        min_idx = (current_idx + 1) % len(self.extracted_clips_array)
        for video_idx in range(len(self.extracted_clips_array)):
            if video_idx == current_idx:
                continue
            clip = self.extracted_clips_array[video_idx].subclip(cur_t, next_t)
            cur_d, plus_frame = self.dist_obj.distance(reference_clip, clip)
            print(current_idx, video_idx, cur_d, cur_t + plus_frame)
            if d > cur_d:
                d = cur_d
                min_idx = video_idx
                next_t = cur_t + plus_frame
                cur_clip = reference_clip.subclip(0, plus_frame)
                
        if cur_clip:
            clip = cur_clip
        else:
            clip = reference_clip
        self.con_clips.append(clip)
        t = next_t
        return t, min_idx
    
    def add_padding(self, t, next_idx):
        print('idx : {}'.format(next_idx))
        pad_clip = self.extracted_clips_array[next_idx].subclip(t, min(self.min_time, t+self.padded_time))
        self.con_clips.append(pad_clip)
        
        t = min(self.min_time, t + self.padded_time)
        return t, next_idx
    
    def write_video(self):
        final_clip = concatenate_videoclips(self.con_clips)
        if self.audioclip != None:
            print('Not None')
            final_clip.audio = self.audioclip
        final_clip.write_videofile(self.output_path)
        return final_clip
    
    def generate_video(self):
        self.video_alignment()
        t = 3
        current_idx = 0
        self.con_clips.append(self.extracted_clips_array[current_idx].subclip(0, min(t, int(self.min_time))))
        while t < int(self.min_time):
            t, min_idx = self.select_next_clip(t, current_idx)
            t, current_idx = self.add_padding(t, min_idx)
        final_clip = self.write_video()
        return final_clip

## face distance 통합

In [2]:
class FaceDistance:
    def __init__(self, shape_predictor_path, face_embedding_penalty = None):
        self.skip_frame_rate = 4
        self.minimax_frames = 5
        self.shape_predictor = shape_predictor_path
        self.face_embedding_penalty = face_embedding_penalty
        
    def extract_landmark(self, reference_clip, compare_clip):
        self.clips = [reference_clip, compare_clip]
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(self.shape_predictor)
        clips_frame_info = []
        for clip in self.clips:
            i = 0
            every_frame_info = []
            while True:
                frame = clip.get_frame(i * 1.0 / clip.fps)
                i += self.skip_frame_rate
                if (i * 1.0 / clip.fps) > clip.duration:
                    break
                    
                frame = imutils.resize(frame, width = 800)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                if len(rects) > 0:
                    max_width = 0
                    max_rect = None
                    for rect in rects:
                        if int(rects[0].width()) > max_width:
                            max_rect = rect
                    shape = predictor(gray, max_rect)
                    shape = face_utils.shape_to_np(shape)
                    every_frame_info.append(shape)
                else : 
                    every_frame_info.append([])
                    
            clips_frame_info.append(np.array(every_frame_info))
        cv2.destroyAllWindows()
        return clips_frame_info
    
    def embedding_cosine_distance(self, reference_frame, compare_frame):
        face_detector = MTCNN(select_largest = True)
        embed_model = InceptionResnetV1(pretrained = 'vggface2').eval()
        
        reference_frame = np.array(reference_frame)
        compare_frame = np.array(compare_frame)
        try:
            reference_frame_detected = face_detector(reference_frame)
            compare_frame_detected = face_detector(compare_frame)
        except:
            cosine_dist = 1
            return cosine_dist
        
        reference_frame_embed = embed_model(reference_frame_detected.unsqueeze(0)).detach().numpy()
        compare_frame_embed = embed_model(compare_frame_detected.unsqueeze(0)).detach().numpy()
        reference_frame_embed = np.squeeze(reference_frame_embed)
        compare_frame_embed = np.squeeze(compare_frame_embed)
        cosine_dist = 1 -\
                    np.dot(reference_frame_embed, compare_frame_embed) /\
                    (np.linalg.norm(reference_frame_embed) *\
                     np.linalg.norm(compare_frame_embed))
        return cosine_dist
    
    def get_all_frame_distance(self, clips_frame_info, min_size):
        dist_arr = []
        for i in range(min_size - 1):
            if len(clips_frame_info[0][i]) > 0 and len(clips_frame_info[1][i+1]):
                l = 36
                r = 45
                left_eye = (
                            (clips_frame_info[0][i][l][0] - clips_frame_info[1][i+1][l][0]) ** 2 +\
                            (clips_frame_info[0][i][l][1] - clips_frame_info[1][i+1][l][1]) ** 2
                           ) ** 0.5
                
                right_eye = (
                             (clips_frame_info[0][i][r][0] - clips_frame_info[1][i+1][r][0]) ** 2+\
                             (clips_frame_info[0][i][r][1] - clips_frame_info[1][i+1][r][1]) ** 2   
                            ) ** 0.5
                total_diff = left_eye + right_eye
                dist_arr.append(total_diff)
            else :
                dist_arr.append(None)
        return dist_arr
    
    def distance(self, reference_clip, compare_clip):
        clips_frame_info = self.extract_landmark(reference_clip, compare_clip)
        min_size = min(len(clips_frame_info[0]), len(clips_frame_info[1]))
        dist_arr = self.get_all_frame_distance(clips_frame_info, min_size)
        clips = [reference_clip, compare_clip]
        minimax_frames = self.minimax_frames
        min_diff = np.float('Inf')
        min_idx = 0
        for i in range(min_size - (minimax_frames - 1)):
            start_minmax_idx = 0 if (i - minimax_frames) < 0 else i - minimax_frames
            if (None not in dist_arr[start_minmax_idx :i + minimax_frames]):
                tmp_max = np.max(dist_arr[start_minmax_idx:i + minimax_frames])
                if min_diff > tmp_max:
                    min_diff = tmp_max
                    min_idx = i
                    
        if self.face_embedding_penalty != None and min_diff < np.float('Inf'):
            ref_frame = reference_clip.get_frame(min_idx * 1.0 / reference_clip.fps)
            frame = compare_clip.get_frame(min_idx * 1.0 / compare_clip.fps)
            cosine_dist = self.embedding_cosine_distance(ref_frame, frame)
            min_diff += cosine_dist * self.face_embedding_penalty
            
        return min_diff, (min_idx * self.skip_frame_rate) / self.clips[0].fps

## Pose distance 통합

In [3]:
class PoseDistance:
    def __init__(self):
        self.SKIP_FRAME_RATE = 10
        self.MINIMAX_FRAME = 4
        self.model = models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
        self.model.eval()
        os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
        
    def extract_boxes(self,reference_clip, compare_clip):
        self.clips = [reference_clip, compare_clip]
        clips_frame_info = []
        for clip in self.clips:
            i = 0
            every_frame_info = []
            while True:
                i += self.SKIP_FRAME_RATE
                if (i * 1.0 / clip.fps) > clip.duration:
                    break
                frame = clip.get_frame(i * 1.0 / clip.fps)
                frame = imutils.resize(frame, width = 640)
                frame = frame / 255
                frame = np.transpose(frame, (2, 0, 1))
                x = [torch.from_numpy(frame).float()]
                predictions = self.model(x)
                prediction = predictions[0]
                each_box_list = zip(prediction['boxes'].tolist(), prediction['labels'].tolist(), prediction['scores'].tolist())
                filtered_box_list = filter(lambda x: x[1] == 1 and x[2] >= 0.95, each_box_list)
                filtered_center_dot_list = list(map(lambda x: [(x[0][0] + x[0][2]) / 2, (x[0][1] + x[0][3]) / 2], filtered_box_list))
                sorted_dot_list = sorted(filtered_center_dot_list, key = lambda x: x[0])
                every_frame_info.append(sorted_dot_list)
                
            clips_frame_info.append(np.array(every_frame_info))
        return clips_frame_info
    
    def get_all_frame_distance(self, clips_frame_info, min_size):
        dist_arr = list()
        for i in range(min_size):
            if len(clips_frame_info[0][i]) > 0 and len(clips_frame_info[1][i]) > 0:
                ref_frame_dots = clips_frame_info[0][i]
                compare_frame_dots = clips_frame_info[1][i]
                min_dot_num = min(len(ref_frame_dots), len(compare_frame_dots))
                dot_num_diff = abs(len(ref_frame_dots) - len(compare_frame_dots))
                penalty = ((self.clips[0].w ** 2 + self.clips[0].h ** 2) ** 0.5) * abs(len(ref_frame_dots) - len(compare_frame_dots))
                total_diff = penalty * dot_num_diff
                for dot_idx in range(min_dot_num):
                    total_diff += (
                        (ref_frame_dots[dot_idx][0] - compare_frame_dots[dot_idx][0]) ** 2 +\
                        (ref_frame_dots[dot_idx][1] - compare_frame_dots[dot_idx][0]) ** 2
                    ) ** 0.5
                dist_arr.append(total_diff)
            else:
                dist_arr.append(None)
        return dist_arr
    
    def distance(self, reference_clip, compare_clip):
        clips_frame_info = self.extract_boxes(reference_clip, compare_clip)
        min_size = min(len(clips_frame_info[0]), len(clips_frame_info[1]))
        dist_arr = self.get_all_frame_distance(clips_frame_info, min_size)
        min_diff = np.float('Inf')
        min_idx = 0
        for i in range(min_size - (self.MINIMAX_FRAME - 1)):
            start_minmax_idx = 0 if (i - self.MINIMAX_FRAME) < 0 else i - self.MINIMAX_FRAME
            if (None not in dist_arr[start_minmax_idx :i + self.MINIMAX_FRAME]):
                tmp_max = np.max(dist_arr[i:i + self.MINIMAX_FRAME])
                if min_diff > tmp_max:
                    min_diff = tmp_max
                    min_idx = i
        return min_diff, (min_idx * self.SKIP_FRAME_RATE) / reference_clip.fps

# 4. 교차편집 실행 코드

### - 1. library import

In [4]:
import os
import random
import datetime
import argparse
import time
import numpy as np
from torchvision import models
import torch.nn as nn
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
import dlib
import cv2
import imutils
from imutils.video import VideoStream
from imutils import face_utils
from moviepy.editor import *
from moviepy.editor import VideoFileClip, concatenate_videoclips


### - 2. 교차편집 실행

In [ ]:
# random, face, pose중에 하나
method = 'pose'
# 무대 영상들이 저장된 폴더
video_path = './ex_video'
# 교차편집 영상을 저장할 경로와 파일명
output_path = './yys_pose.mp4'
# 얼굴의 landmark를 추출할 때 사용하는 predictor의 경로
shape_predictor_path = 'shape_predictor_68_face_landmarks.dat'
# 얼굴의 embedding 값으로 penalty를 줄지. 값이 클 수록 강한 penalty
face_embedding_penalty = 100 # or None
# 선택한 method에 따라 거리 측정 객체를 생성하고 해당 객체를 전달 해 교차편집 객체 cross_cut 생성
# 교차편집 생성함수 generate_video()를 이용해 교차편집을 만들어 저장
print(output_path)
if method == 'random':
    random_distance = RandomDistance()
    cross_cut = Crosscut(random_distance, video_path, output_path)
elif method == 'face':
    face_distance = FaceDistance(shape_predictor_path, face_embedding_penalty)
    cross_cut = Crosscut(face_distance, video_path, output_path)
elif method == 'pose':
    pose_distance = PoseDistance()
    cross_cut = Crosscut(pose_distance, video_path, output_path)

cross_cut.generate_video()


./yys_pose.mp4
Logger-- 5 Video Will Be Mixed
0 1 202.20843159761407 8.672333333333334
0 2 inf 3.0
0 3 inf 3.0
0 4 inf 3.0
idx : 1
1 0 166.92133982568072 19.012
1 2 163.9601369913043 17.343666666666667
1 3 inf 12.672333333333334
1 4 inf 12.672333333333334
idx : 2
2 0 93.00789344259424 22.678333333333335
